In [1]:
import matplotlib.pyplot as plt
import numpy as np
import astropy.units as u
from datetime import datetime
from astropy.coordinates import SkyCoord
import os
import pickle
import json
import pandas as pd
pd.set_option('display.max_columns', None)

from lstchain.reco.utils import  get_effective_time

from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# dirs
objects_dir = "/fefs/aswg/workspace/juan.jimenez/systematics/objects"


In [2]:
###################
# reading objects
###################
fname_dict = "objects/total_dict_runs.json"
with open(fname_dict, "r") as json_file:
    dict_runs = json.load(json_file)
    
dict_runs["3319"]

{'dl2': '/fefs/aswg/workspace/abelardo.moralejo/Crab_performance_paper/data_v0.9.9/DL2/process_with_mc_theta_10.0_az_102.199/dl2_LST-1.Run03319.h5',
 'irf': '/fefs/aswg/workspace/juan.jimenez/data/systematics/irf/irf_mc_theta_10.0_az_102.199.fits.gz',
 'dl3': '/fefs/aswg/workspace/juan.jimenez/data/systematics/dl3/dl3_LST-1.Run03319.fits.gz'}

In [3]:
%%time
dict_dl2 = {}

for i, run in enumerate(dict_runs.keys()):
    
    print(f"Iterating... {i/len(dict_runs.keys())*100:4.1f}%")
    
    _tab = pd.read_hdf(dict_runs[run]["dl2"], key='dl2/event/telescope/parameters/LST_LSTCam')
    
    # getting only cosmics and applying a small cut in gammaness
    _tab.query("event_type == 32 and gammaness > 0.05", inplace=True)
    
    eff_t, el_t = get_effective_time(_tab)
    
    tmp_dict = {
        "effective_time" : eff_t.to(u.s).value,
        "elapsed_time" : el_t.to(u.s).value,
        "zd" : np.array(90 - np.rad2deg(_tab["alt_tel"])),
        "az" : np.array(np.rad2deg(_tab["az_tel"])),
        "intensity" : np.array(_tab["intensity"]),
        "gammaness" : np.array(_tab["gammaness"])
    }

    dict_dl2[int(run)] = tmp_dict

Iterating...  0.0%
Iterating...  0.9%
Iterating...  1.7%
Iterating...  2.6%
Iterating...  3.4%
Iterating...  4.3%
Iterating...  5.1%
Iterating...  6.0%
Iterating...  6.8%
Iterating...  7.7%
Iterating...  8.5%
Iterating...  9.4%
Iterating... 10.3%
Iterating... 11.1%
Iterating... 12.0%
Iterating... 12.8%
Iterating... 13.7%
Iterating... 14.5%
Iterating... 15.4%
Iterating... 16.2%
Iterating... 17.1%
Iterating... 17.9%
Iterating... 18.8%
Iterating... 19.7%
Iterating... 20.5%
Iterating... 21.4%
Iterating... 22.2%
Iterating... 23.1%
Iterating... 23.9%
Iterating... 24.8%
Iterating... 25.6%
Iterating... 26.5%
Iterating... 27.4%
Iterating... 28.2%
Iterating... 29.1%
Iterating... 29.9%
Iterating... 30.8%
Iterating... 31.6%
Iterating... 32.5%
Iterating... 33.3%
Iterating... 34.2%
Iterating... 35.0%
Iterating... 35.9%
Iterating... 36.8%
Iterating... 37.6%
Iterating... 38.5%
Iterating... 39.3%
Iterating... 40.2%
Iterating... 41.0%
Iterating... 41.9%
Iterating... 42.7%
Iterating... 43.6%
Iterating...

In [4]:
%%time
# Saving the objects
with open('objects/dict_dl2.pkl', 'wb') as f:
    pickle.dump(dict_dl2, f, pickle.HIGHEST_PROTOCOL)
    
# # this to read the objects
# with open('objects/dict_dl2.pkl', 'rb') as f:
#     dict_dl2 = pickle.load(f)

CPU times: user 563 ms, sys: 27.8 s, total: 28.3 s
Wall time: 2min 7s
